In [ ]:
"""
You can run either this notebook locally (if you have all the dependencies and a GPU) or on Google Colab.

Instructions for setting up Colab are as follows:
1. Open a new Python 3 notebook.
2. Import this notebook from GitHub (File -> Upload Notebook -> "GITHUB" tab -> copy/paste GitHub URL)
3. Connect to an instance with a GPU (Runtime -> Change runtime type -> select "GPU" for hardware accelerator)
4. Run this cell to set up dependencies.
5. Restart the runtime (Runtime -> Restart Runtime) for any upgraded packages to take effect
"""

NEMO_DIR_PATH = "NeMo"
BRANCH = 'r1.22.0'

! git clone https://github.com/NVIDIA/NeMo
%cd NeMo
! python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[all]
%cd ..

# Speaker Diarization Training

## Neural Diarizer in Speaker Diarization Pipeline

<img src="images/diar_pipeline.png" alt="diar_pipeline" style="width: 800px;"/>

Speaker diarization system needs to capture the characteristics of unseen speakers from the given audio recording and generate speaker-homogeneous segments which belong to corresponding speaker labels. During the speaker diarization process, the number of speakers should be estimated, then the audio segments should be assigned to a few of speaker labels. 

While clustering algorithms can also assign segments to speaker groups, overlap-aware diarization cannot be done with clustering based diarizer since one segment is only assigned to one speaker label. However, we can use the clustering result to create initial speaker profiles and train a neural model that generates overlap-aware speaker labels by comparing the input audio signal with the initial speaker profiles. In the NeMo speaker diarization toolkit, we refer to such neural modules as **neural diarizer**. 

The Multi-scale Diarization Decoder (MSDD) model is a type of neural diarizer we can use in the NeMo speaker diarization pipeline. This tutorial shows how to train MSDD on a small toy dataset. By using MSDD on top of clustering diarizer, we can obtain the following benefits:

- **Improved diarization accuracy**: Compared to clustering diarizer, MSDD could achieve a lower diarization error rate (DER)
- **Overlap aware diarization**: Speaker diarization results in clustering diarizer do not include speech overlaps
- **Model training on actual multispeaker dataset**: Unlike training a speaker embedding model, we can train or finetune a neural model on an actual speaker diarization dataset where multiple speakers are recorded in a single audio file. 

### Training and inference of Multi-scale Diarization Decoder

When it comes to the speaker diarization problem, MSDD model employs a divide-and-conquer strategy where a pairwise model is employed for both training and inference. The following figure explains how a pairwise model is employed for training and inference.

<img src="images/msdd_train_and_infer.png" alt="MSDD_train_infer" style="width: 800px;"/>

Here are itemized descriptions of noteworthy features of the MSDD model.

#### Training 

- **Oracle VAD, multi-scale segmentation for training**   
In a training setup, we use oracle VAD from ground-truth annotation files (RTTM files) and perform multiscale segmentation. After we obtain timestamps for each and every segment, we feed multi-scale timestamps and raw audio signals into a computational graph where the speaker embedding extractor and neural diarizer is trained.  

   
- **MSDD inputs for training process**  
During training, we employ oracle clustering result (ground-truth speaker labels in the annotation file) to calculate the cluster-average embeddings. Subsequently, we calculate binary cross-entropy loss which calculates a loss value for each timestep and each speaker.


- **End-to-end training: from raw audio to speaker label**   
The training approach we employ can be considered as end-to-end training since the input to the computational graph is raw audio signal and the outputs are speaker labels. The end-to-end training is depicted in a dotted box in the above figure. We can either freeze the speaker embedding model or train it jointly depending on the tasks.   


- **Pairwise (two-speaker) unit model**   
While training the MSDD model, we use a two-speaker dataset for a two speaker model. For this pairwise training, we clean the source dataset to have only two speakers by splitting the annotation.   


- **Split training samples**  
Since we have finite GPU memory for training, we break down the training audio samples into short audio samples. We set step-count, and step-count indicates a unit of decision for speaker label estimation. We set step-count (e.g., `step_count=50`) when we create training datasets and use the step-count for training.    

#### Inference

- **Multi-scale clustering**  
In inference mode, we apply multi-scale clustering for obtaining speaker profiles that are represented by cluster-average embedding.   


- **Divide-and-conquer approach with pairwise (two-speaker) unit model**   
We retrieve all possible pairs from the estimated number of speakers and average the results. For example, if there are four speakers `(A, B, C, D)`, we extract 6 pairs: `(A,B)`, `(A,C)`, `(A,D)`, `(B,C)`, `(B,D)`, `(C,D)`. Finally, the sigmoid outputs are averaged. In this way, MSDD can deal with a flexible number of speakers using a pairwise model.


- **Split inference samples**  
As in the training process, we can also break down the target samples for inference. While we can do inference on whole input audio at once, split inference generally gives an improved performance. It is recommended to use the same step-count you used for training the MSDD model (e.g., `diar_window_length=50`) for your inference configurations.

### Input and Output of Multi-scale Diarization Decoder

While using an MSDD model as neural diarizer has a few benefits, MSDD models require a clustering result to obtain initial speaker profiles as references for performing overlap-aware speaker diarization inference.  Here are descriptions for input and output of the MSDD model.

#### Input: Clustering as Initialization

MSDD model is a diarizer model that accepts two different data inputs:

 1. Cluster-average embeddings
 2. Multi-scale embedding sequence 
 
The two input signals are depicted in the following figure.

<img src="images/msdd_inputs.png" alt="MSDD_inputs" style="width: 600px;"/>

By initializing the diarization task with a clustering algorithm, we can estimate the number of speakers and cluster-average embeddings. Thus, the cluster-average embeddings provide the speaker profile of each speaker. The cluster-average embeddings we provide can be regarded as reference signals for providing seed speaker profiles.

Once we obtain the fixed (or estimated) number of speakers, the speaker diarization problem becomes a binary classification task where we need to estimate whether a certain speaker's speech exists or not at a given timestep.  

#### Output: Sigmoid Output and Binary Cross-entropy Loss

<img src="images/msdd_output_loss.png" alt="MSDD_output_loss" style="width: 600px;"/>

The above figure depicts the inputs and outputs of the MSDD model. As an output from MSDD, sigmoid values for each speaker are generated. These sigmoid values are independent from the other speakers and indicate the simulated probability of the corresponding speaker's speech signal at the given step. During the training process, binary cross-entropy (BCE) is calculated for each individual sigmoid value and summed up to calculate the total loss for optimization.

## Example Data Creation

- Please skip this section and go directly to [Prepare Training data for MSDD](#Prepare-Training-data-for-MSDD) section if you have your own speaker diarization dataset. 

In this tutorial, we use [NeMo Multispeaker Simulator](https://github.com/NVIDIA/NeMo/blob/main/tutorials/tools/Multispeaker_Simulator.ipynb) and the Librispeech corpus to generate a toy training dataset for demonstration purpose. You can replace the simulated dataset with your own datasets if you have proper speaker annotations (RTTM files) for the dataset. If you do not have access to any speaker diarization datasets, you can use [NeMo Multispeaker Simulator](https://github.com/NVIDIA/NeMo/blob/main/tutorials/tools/Multispeaker_Simulator.ipynb) by generating a good amount of data samples to meet your needs. 

For more details regarding data simulator, please follow the descriptions in [NeMo Multispeaker Simulator](https://github.com/NVIDIA/NeMo/blob/main/tutorials/tools/Multispeaker_Simulator.ipynb) and we will not cover configurations and detailed process of data simulation in this tutorial. 


In [ ]:
# Install dependencies for data simulator
!apt-get install sox libsndfile1 ffmpeg
!pip install wget
!pip install unidecode
!pip install "matplotlib>=3.3.2"

#### Data Simulation Step 1:  Download Required Resources

We need to download the LibriSpeech corpus and corresponding word alignments for generating synthetic multi-speaker audio sessions. In addition, we need to download necessary data cleaning scripts from NeMo git.

In [ ]:
import os
NEMO_DIR_PATH = "NeMo"
BRANCH = 'r1.22.0'

# download scripts if not already there 
if not os.path.exists('NeMo/scripts'):
  print("Downloading necessary scripts")
  !mkdir -p NeMo/scripts/dataset_processing
  !mkdir -p NeMo/scripts/speaker_tasks
  !wget -P NeMo/scripts/dataset_processing/ https://raw.githubusercontent.com/NVIDIA/NeMo/$BRANCH/scripts/dataset_processing/get_librispeech_data.py
  !wget -P NeMo/scripts/speaker_tasks/ https://raw.githubusercontent.com/NVIDIA/NeMo/$BRANCH/scripts/speaker_tasks/create_alignment_manifest.py
  !wget -P NeMo/scripts/speaker_tasks/ https://raw.githubusercontent.com/NVIDIA/NeMo/$BRANCH/scripts/speaker_tasks/create_msdd_train_dataset.py  
  !wget -P NeMo/scripts/speaker_tasks/ https://raw.githubusercontent.com/NVIDIA/NeMo/$BRANCH/scripts/speaker_tasks/pathfiles_to_diarize_manifest.py

Now that we have downloaded all the necessary scripts for data creation and preparation, we can start the data simulation step by downloading the LibriSpeech corpus.

In [ ]:
!mkdir -p LibriSpeech
!python {NEMO_DIR_PATH}/scripts/dataset_processing/get_librispeech_data.py \
  --data_root LibriSpeech \
  --data_sets dev_clean

We can get the forced word alignments data for the LibriSpeech corpus from [this repository.](https://github.com/CorentinJ/librispeech-alignments). Full forced alignments data can be downloaded at [google drive link for alignments data](https://drive.google.com/file/d/1WYfgr31T-PPwMcxuAq09XZfHQO5Mw8fE/view?usp=sharing). We will download only a subset of forced alignment data containing dev-clean part.

In [ ]:
!wget -nc https://dldata-public.s3.us-east-2.amazonaws.com/LibriSpeech_Alignments.tar.gz
!tar -xzf LibriSpeech_Alignments.tar.gz
!rm -f LibriSpeech_Alignments.tar.gz

#### Data Simulation Step 2:  Produce Manifest File with Forced Alignments

We will merge the LibriSpeech manifest files and LibriSpeech forced alignments into one manifest file for ease of use when generating synthetic data. Create alignment files by running the following script.



In [ ]:
!python NeMo/scripts/speaker_tasks/create_alignment_manifest.py \
  --input_manifest_filepath LibriSpeech/dev_clean.json \
  --base_alignment_path LibriSpeech_Alignments \
  --output_manifest_filepath ./dev-clean-align.json \
  --ctm_output_directory ./ctm_out \
  --libri_dataset_split dev-clean

#### Data Simulation Step 3:  Set data simulation parameters

Now that we have downloaded all the sources we need for data creation, we need to download data simulator configurations in `.yaml` format. Download the YAML file and download `data_simulator.py` script from NeMo repository.

In [ ]:
from omegaconf import OmegaConf
import os
ROOT = os.getcwd()
conf_dir = os.path.join(ROOT,'conf')
!mkdir -p $conf_dir
CONFIG_PATH = os.path.join(conf_dir, 'data_simulator.yaml')
if not os.path.exists(CONFIG_PATH):
  !wget -P $conf_dir https://raw.githubusercontent.com/NVIDIA/NeMo/$BRANCH/tools/speech_data_simulator/conf/data_simulator.yaml

config = OmegaConf.load(CONFIG_PATH)
print(OmegaConf.to_yaml(config))

#### Data Simulation Step 4:   Generate Simulated Audio Session

We will generate a set of example sessions with the following specifications:

- 3 example sessions for train  
- 3 example sessions for validation
- 2-speakers in each session
- 60 seconds of recordings

We need to setup different seed for train and validation sets.

In [ ]:
from nemo.collections.asr.data.data_simulation import MultiSpeakerSimulator

# Generate train set 
ROOT = os.getcwd()
data_dir = os.path.join(ROOT,'simulated_train')
config.data_simulator.random_seed=10
config.data_simulator.manifest_filepath="./dev-clean-align.json"
config.data_simulator.outputs.output_dir=data_dir
config.data_simulator.session_config.num_sessions=3
config.data_simulator.session_config.num_speakers=2
config.data_simulator.session_config.session_length=60
config.data_simulator.background_noise.add_bg=False 

lg = MultiSpeakerSimulator(cfg=config)
lg.generate_sessions()

# Generate validation set 
data_dir = os.path.join(ROOT,'simulated_valid')
config.data_simulator.random_seed=20
config.data_simulator.outputs.output_dir=data_dir

lg = MultiSpeakerSimulator(cfg=config)
lg.generate_sessions()

Now that parameter setting is done, generate the samples by launching `generate_sessions()`.

In [ ]:
lg = MultiSpeakerSimulator(cfg=config)
lg.generate_sessions()

### Data preparation step 5: Listen to and Visualize Session

Listen to the audio and visualize the corresponding speaker timestamps (recorded in a RTTM file for each session).

In [ ]:
import os
import wget
import IPython
import matplotlib.pyplot as plt
import numpy as np
import librosa
from nemo.collections.asr.parts.utils.speaker_utils import rttm_to_labels, labels_to_pyannote_object

ROOT = os.getcwd()
data_dir = os.path.join(ROOT,'simulated_train')
audio = os.path.join(data_dir,'multispeaker_session_0.wav')
rttm = os.path.join(data_dir,'multispeaker_session_0.rttm')

sr = 16000
signal, sr = librosa.load(audio,sr=sr) 

fig,ax = plt.subplots(1,1)
fig.set_figwidth(20)
fig.set_figheight(2)
plt.plot(np.arange(len(signal)),signal,'gray')
fig.suptitle('Synthetic Audio Session', fontsize=16)
plt.xlabel('Time (s)', fontsize=18)
plt.yticks([], [])
ax.margins(x=0)
a,_ = plt.xticks()
plt.xticks(a[:-1],a[:-1]/sr);
IPython.display.Audio(audio)

We can visually check the ground-truth file of the first sample by running the following commands. We can see that it has plenty of overlap between two speakers. 

In [ ]:
# display speaker labels for reference
labels = rttm_to_labels(rttm)
reference = labels_to_pyannote_object(labels)
reference

You can check that corresponding RTTM files are generated as ground-truth labels for training and evaluation.

In [ ]:
!cat simulated_train/multispeaker_session_0.rttm | head -10

### Data preparation step 6: Check out the created files

The following files are generated from data simulator:

* _wav files_ (one per audio session) - the output audio sessions
* _rttm files_ (one per audio session) - the speaker timestamps for the corresponding audio session (used for diarization training)
* _list files_ (one per file type per batch of sessions) - a list of generated files of the given type (e.g., wav, rttm), used primarily for manifest creation

Check if the files we need are generated by running the following commands.

In [ ]:
print("\n Training audio files:")
!ls simulated_train/*.wav
print("\n Training audio files:")
!ls simulated_train/*.rttm
print("\n Training RTTM list content:")
!cat simulated_train/synthetic_wav.list
print("\n Training RTTM list content:")
!cat simulated_train/synthetic_rttm.list

print("\n Validation audio files:")
!ls simulated_valid/*.wav
print("\n Validation audio files:")
!ls simulated_valid/*.rttm
print("\n Validation RTTM list content:")
!cat simulated_valid/synthetic_wav.list
print("\n Validation RTTM list content:")
!cat simulated_valid/synthetic_rttm.list

## Prepare Training Data for MSDD

Now that we have datasets for both train and validation (dev), we can start preparing and cleaning the data samples for training. Make sure you have the following list of files:

**Training set** 

- Train audio files `.wav`
- A train audio list file `.list`
- Train RTTM files `.rttm`
- A train RTTM list content `.list`

**Validation set**  

- Validation audio files `.wav`
- A validation audio list file `.list`
- Validation RTTM files `.rttm`
- A validation RTTM list file `.list`


Based on these files, we need to create manifest files containing data samples we have. If you don't have a `.list` file, you need to create a `.list` file for the `.wav` files and `.rttm` files.

In [ ]:
# create a NeMo manifest (.json) file for training dataset
!python NeMo/scripts/speaker_tasks/pathfiles_to_diarize_manifest.py \
  --paths2audio_files='simulated_train/synthetic_wav.list' \
  --paths2rttm_files='simulated_train/synthetic_rttm.list' \
  --manifest_filepath='simulated_train/msdd_data.json'

# create a NeMo manifest (.json) file for validation (dev) dataset
!python NeMo/scripts/speaker_tasks/pathfiles_to_diarize_manifest.py \
  --paths2audio_files='simulated_valid/synthetic_wav.list' \
  --paths2rttm_files='simulated_valid/synthetic_rttm.list' \
  --manifest_filepath='simulated_valid/msdd_data.json'

If you print the content of the created manifest file, you can see that `.rttm` files in the list and `.wav` files are grouped together in the generated manifest files.

In [ ]:
print("\n An example line in training dataset manifest file:")
!cat simulated_train/msdd_data.json | head -1
print("\n An example line in validation Dataset manifest file:")
!cat simulated_valid/msdd_data.json | head -1

Now that we have input a standard manifest file, we need to break down each audio clip into short audio clips so that we can put several samples in a batch. 

Before we generate a manifest file and RTTM files for training MSDD, you have to determine:

- `window`: the window length of the base scale (the shortest scale)
- `shift`: the hop-length of the base scale (the shortest scale)
- `step_count`: how many decision steps in one data sample

Note that these numbers should match the parameters in the configurations for your desired MSDD model. If you want to train with new parameters (`window`, `shift` and `step_count`), you need to make new manifest files with the new parameters.

In [ ]:
# create a manifest (.json) file for training dataset
!python NeMo/scripts/speaker_tasks/create_msdd_train_dataset.py \
  --input_manifest_path='simulated_train/msdd_data.json' \
  --output_manifest_path='simulated_train/msdd_data.50step.json' \
  --pairwise_rttm_output_folder='simulated_train/' \
  --window 0.5 \
  --shift 0.25 \
  --step_count 50 
  
# create a manifest (.json) file for validation (dev) dataset
!python NeMo/scripts/speaker_tasks/create_msdd_train_dataset.py \
  --input_manifest_path='simulated_valid/msdd_data.json' \
  --output_manifest_path='simulated_valid/msdd_data.50step.json' \
  --pairwise_rttm_output_folder='simulated_valid/' \
  --window 0.5 \
  --shift 0.25 \
  --step_count 50 

Now that we broke down the training and validation dataset into 50-step samples, let's checkout how the output manifest files look like. We used 0.25 second of shift length so in theory, if there is no silence or pause in the data, the length of data sample should be `step_count*shift` which is `50*0.25=12.5` second in the example we used. However, since there are pauses between the segments in practice, the final lengths of data samples are longer than 12.5 second.

In [ ]:
print("\nTraining Dataset:")
!cat simulated_train/msdd_data.50step.json | tail -5
print("\nValidation Dataset:")
!cat simulated_valid/msdd_data.50step.json | tail -5 

## Train an MSDD Model

Now that we have prepared all the necessary dataset, we can train an MSDD model on the prepared dataset. Download YAML file for training form NeMo repository and load the configuration into `config` variable.

In [ ]:
import nemo
import nemo.collections.asr as nemo_asr
from omegaconf import OmegaConf

NEMO_ROOT = os.getcwd()
!mkdir conf 
!wget -P conf https://raw.githubusercontent.com/NVIDIA/NeMo/$BRANCH/examples/speaker_tasks/diarization/conf/neural_diarizer/msdd_5scl_15_05_50Povl_256x3x32x2.yaml
MODEL_CONFIG = os.path.join(NEMO_ROOT,'conf/msdd_5scl_15_05_50Povl_256x3x32x2.yaml')
config = OmegaConf.load(MODEL_CONFIG)
print(OmegaConf.to_yaml(config))

Setup the `manifest_filepath` for `train_ds` and `validation_ds` by feeding the `json` file paths from `create_msdd_train_dataset.py`.

In [ ]:
config.model.train_ds.manifest_filepath = 'simulated_train/msdd_data.50step.json'
config.model.validation_ds.manifest_filepath = 'simulated_valid/msdd_data.50step.json'

### Train MSDD with frozen speaker embedding model

Provide a batch size number for training in `config.model.batch_size`. In a batch, we will assign the given number of split samples (in this example, 50 step-size). Note that you might need to change this batch size if the following batch size maxes out your GPU memory size. 

`config.model.emb_batch_size` determines the number of embedding vectors attached to a computational graph. This means that 
If you want to freeze the speaker embedding extractor, you should set `emb_batch_size=0`.
If you want to jointly optimize speaker embedding extractor, you need to assign an adequate number that does not max out the GPU memory. 

In [ ]:
config.batch_size=5
config.model.emb_batch_size=0

Provide paths to the temporary folders for saving timestamp data during training.

In [ ]:
config.model.train_ds.emb_dir="simulated_train" 
config.model.validation_ds.emb_dir="simulated_valid" 

Setup a speaker embedding model that will be used for speaker embedding extraction. We will use `titanet_large` model checkpoint from NGC. Note that this speaker embedding model will be saved together in a `.ckpt` file whenever pytorch lightning trainer saves checkpoint. 

In [ ]:
config.model.diarizer.speaker_embeddings.model_path="titanet_large"
config.trainer.max_epochs = 5
config.trainer.strategy = 'auto'

We will use `pytorch_lightning` and train a model instance from class `EncDecDiarLabelModel`.

In [ ]:
import pytorch_lightning as pl
from nemo.collections.asr.models import EncDecDiarLabelModel
from nemo.utils.exp_manager import exp_manager

trainer = pl.Trainer(**config.trainer)
exp_manager(trainer, config.get("exp_manager", None))
msdd_model = EncDecDiarLabelModel(cfg=config.model, trainer=trainer)

Before we start training, let's check a few of the weights in the speaker embedding model in `msdd_model.msdd._speaker_model`.

In [ ]:
msdd_model.msdd._speaker_model.state_dict()["encoder.encoder.0.mconv.0.conv.weight"][0,:,:]

If you successfully ran the previous step, now it is ready to initiate a training session of MSDD. Launch `trainer.fit()` function for `msdd_model`. 

In [ ]:
trainer.fit(msdd_model)

In this way, you can train an MSDD model and use `.ckpt` files saved during training process.

Let's check the weights in speaker embedding model again and see if the numbers are changed.

In [ ]:
msdd_model.msdd._speaker_model.state_dict()["encoder.encoder.0.mconv.0.conv.weight"][0,:,:]

### Train MSDD and speaker embedding model together

In this time, let's attach a few speaker embeddings to a graph and jointly train the speaker embedding mode.

In [ ]:
config.model.emb_batch_size = 100 # choose the largest number that does not max out GPU memory.
config.trainer.max_epochs = 5

Setup another trainer and initiate a training session with the new parameters.

In [ ]:
trainer = pl.Trainer(**config.trainer)
exp_manager(trainer, config.get("exp_manager", None))
msdd_model = EncDecDiarLabelModel(cfg=config.model, trainer=trainer)
trainer.fit(msdd_model)

Let's check whether there is a change in the weights.

In [ ]:
msdd_model.msdd._speaker_model.state_dict()["encoder.encoder.0.mconv.0.conv.weight"][0,:,:]